<a href="https://colab.research.google.com/github/Riddles1124/wagonden/blob/main/Engineering_Pipeline_Modules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow
import tensorflow.keras as keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras import models
from keras import layers
from keras import optimizers

from sklearn.model_selection import train_test_split as tts

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

IMAGE_SIZE = [200,200,3]
con_base = VGG16(weights='imagenet',include_top=False,input_shape=IMAGE_SIZE)
con_base.summary()

db_r = '/content/drive/Othercomputers/My MacBook Air/Deep Learning/Database/'
# os.mkdir(db_home)
train_dir = os.path.join(db_r, 'train')
# os.mkdir(train_dir)
validation_dir = os.path.join(db_r, 'validation')
# os.mkdir(validation_dir)
test_dir = os.path.join(db_r, 'test')
# os.mkdir(test_dir)
images = os.path.join(db_r, 'images')
# os.mkdir(images)



In [ ]:

def local_database_creator(names, file_path_dict):
    conn = sqlite3.connect('dog_image_database')
    c = conn.cursor()
    breeds = [name for name in names]
    for breed in breeds:
        c.execute(f"CREATE TABLE IF NOT EXISTS {breed} ([FILEPATH] TEXT)")
    conn.commit()
    db = database_creator(file_path_dict)
    for ind, bname in enumerate(file_path_dict):
        db[ind].to_sql(f'{bname}', conn, if_exists = 'append')
    return print("Database Created")

breed_folder_names = []
for val in names:
  breed_folder_names.append(os.path.join(db_home,val))


def dict_maker_two(names,breed_folder_names):
  dogedict = {}
  for ind, name in enumerate(names):
    if name == "Japanese_Spitzes":
      dogedict[name] = [val for val in os.walk(breed_folder_names[ind])]
      dogedict[name] = dogedict[name][1][2]
    else:
      dogedict[name] = [val for val in os.walk(breed_folder_names[ind])]
      dogedict[name] = dogedict[name][2][2]
  return dogedict

def split_dictionary_creator(picdict):
  split_dict = {}
  for val in picdict.keys():
      trainx, testx = tts(picdict[val][1:], test_size = .25, random_state=321)
      split_dict[val] = (trainx,testx)
  return split_dict  

split_dict = split_dictionary_creator(picdict)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(200, 200),
    batch_size=100,
    class_mode='categorical')


test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(200,200),
    batch_size=35,
    class_mode='categorical')

model = models.Sequential()
model.add(con_base)
model.add(layers.Flatten())
model.add(layers.Dense(504, activation='LeakyReLU'))
model.add(layers.Dropout(.5))
model.add(layers.Dense(252, activation='LeakyReLU'))
model.add(layers.Dense(126, activation='softmax'))

# model.summary()

con_base.trainable = False

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(
      train_generator,
      steps_per_epoch=156,
      epochs=50,
      validation_data=test_generator,
      validation_steps=5,
      verbose=1)

with open((db_r+'/history.pkl'), 'wb') as h:
  pickle.dump(history, h)

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(accuracy))

plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()